# ISIBrnoAIMT Encoder with Attention Decoder

Encoder was taken from the winner of the [Will Two Do?](https://physionet.org/content/challenge-2021/1.0.3/sources/) challenge [ISIBrnoAIMT](https://www.cinc.org/archives/2021/pdf/CinC2021-014.pdf)
Decoder was taken from the [sequence to sequence tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html) from Pytorch.

In [1]:
import pickle
import os
import sys
import torch
import pandas as pd
from sklearn.metrics import f1_score, jaccard_score, confusion_matrix, precision_score, recall_score, accuracy_score

from models.m04_EcgToText_ISIBrnoAIMT.dataset import *
from models.m04_EcgToText_ISIBrnoAIMT.model import *
from models.m04_EcgToText_ISIBrnoAIMT.train import *

In [2]:
os.chdir('..')

## Train with different Encoder hidden size

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

language, dataloader = get_dataloader(file_path='./data_ptb-xl', batch_size=64, mode='train', device=device)
_, val_dataloader = get_dataloader(file_path='./data_ptb-xl', batch_size=64, mode='val', device=device, _lang=language)
    
n_epochs=50
encoder_hidden = [128, 256, 512, 1024, 2048]
decoder_hidden_size = 256
criterion = nn.NLLLoss()

for encoder_hidden_size in encoder_hidden:
    encoder = NN(num_leads=12,
                 hidden_size=encoder_hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size=decoder_hidden_size,
                             encoder_hidden_size=encoder_hidden_size,
                             output_size=language.n_words,
                             max_len=language.max_len).to(device)
    
    print(f"############################ encoder hidden size: {encoder_hidden_size} ############################")
    print(f"encoder #parameters: {count_parameters(encoder)}")
    print(f"decoder #parameters: {count_parameters(decoder)}\n")

    train(dataloader, val_dataloader, encoder, decoder, criterion, language, n_epochs, size=encoder_hidden_size)

############################ encoder hidden size: 128 ############################
encoder #parameters: 1650560
decoder #parameters: 2055397
0m 23s (- 19m 9s) (1 2.0%) | Train Loss: 0.5922 | Val METEOR: 0.3593
0m 41s (- 16m 47s) (2 4.0%) | Train Loss: 0.2485 | Val METEOR: 0.3862
1m 0s (- 15m 46s) (3 6.0%) | Train Loss: 0.2114 | Val METEOR: 0.4228
1m 18s (- 15m 5s) (4 8.0%) | Train Loss: 0.1902 | Val METEOR: 0.3936
1m 37s (- 14m 33s) (5 10.0%) | Train Loss: 0.1738 | Val METEOR: 0.4358
1m 55s (- 14m 6s) (6 12.0%) | Train Loss: 0.1627 | Val METEOR: 0.4383
2m 13s (- 13m 42s) (7 14.0%) | Train Loss: 0.1526 | Val METEOR: 0.4161
2m 32s (- 13m 19s) (8 16.0%) | Train Loss: 0.1451 | Val METEOR: 0.4464
2m 50s (- 12m 57s) (9 18.0%) | Train Loss: 0.1383 | Val METEOR: 0.447
3m 9s (- 12m 36s) (10 20.0%) | Train Loss: 0.1333 | Val METEOR: 0.4418
3m 27s (- 12m 15s) (11 22.0%) | Train Loss: 0.128 | Val METEOR: 0.457
3m 45s (- 11m 54s) (12 24.0%) | Train Loss: 0.1229 | Val METEOR: 0.4685
4m 4s (- 11m 34s

RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Test

In [9]:
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder_hidden = [128, 256, 512, 1024, 2048]
decoder_hidden_size = 256
criterion = nn.NLLLoss()

language, dataloader = get_dataloader(file_path='./data_ptb-xl', batch_size=64, mode='train', device=device)
_, test_dataloader = get_dataloader(file_path='./data_ptb-xl', batch_size=64, mode='test', device=device, _lang=language)
results = []

# for encoder_hidden_size in encoder_hidden:
for encoder_hidden_size in [128, 256, 512]:
    encoder = NN(num_leads=12,
                 hidden_size=encoder_hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size=decoder_hidden_size,
                             encoder_hidden_size=encoder_hidden_size,
                             output_size=language.n_words,
                             max_len=language.max_len).to(device)
    
    encoder.load_state_dict(torch.load(f'./models/m04_EcgToText_ISIBrnoAIMT/models_with_different_hidden_size/varying_encoder/Encoder_{encoder_hidden_size}.pth'))
    decoder.load_state_dict(torch.load(f'./models/m04_EcgToText_ISIBrnoAIMT/models_with_different_hidden_size/varying_encoder/Decoder_{encoder_hidden_size}.pth'))
    
    total_loss, f1, jaccard, rouge, meteor = validate_epoch(test_dataloader, encoder, decoder, criterion, language)

    results.append({
        "Encoder": f"Encoder {encoder_hidden_size}",
        "Encoder #parameters": count_parameters(encoder),
        "Decoder #parameters": count_parameters(decoder),
        "Test Loss": round(total_loss, 4),
        "Rouge-1 (p)": round(rouge["rouge-1"]["p"], 3),
        "Rouge-1 (r)": round(rouge["rouge-1"]["r"], 3),
        "Rouge-1 (f1)": round(rouge["rouge-1"]["f"], 3),
        "Rouge-2 (p)": round(rouge["rouge-2"]["p"], 3),
        "Rouge-2 (r)": round(rouge["rouge-2"]["r"], 3),
        "Rouge-2 (f1)": round(rouge["rouge-2"]["f"], 3),
        "Rouge-L (p)": round(rouge["rouge-l"]["p"], 3),
        "Rouge-L (r)": round(rouge["rouge-l"]["r"], 3),
        "Rouge-L (f1)": round(rouge["rouge-l"]["f"], 3),
        "METEOR": round(meteor, 3)
    })

df_results = pd.DataFrame(results)
df_results

,Encoder,Encoder #parameters,Decoder #parameters,Test Loss,Rouge-1 (p),Rouge-1 (r),Rouge-1 (f1),Rouge-2 (p),Rouge-2 (r),Rouge-2 (f1),Rouge-L (p),Rouge-L (r),Rouge-L (f1),METEOR
0,Encoder 128,1650560,2055397,3.6968,0.581,0.600,0.566,0.440,0.451,0.425,0.578,0.596,0.562,0.495
1,Encoder 256,6545152,2219237,3.4800,0.587,0.606,0.572,0.447,0.459,0.433,0.583,0.601,0.568,0.501
2,Encoder 512,26066432,2546917,2.6557,0.507,0.502,0.476,0.346,0.324,0.313,0.502,0.498,0.472,0.391


## Train with different Decoder hidden size

In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [7]:
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

language, dataloader = get_dataloader(file_path='./data_ptb-xl', batch_size=64, mode='train', device=device)
_, val_dataloader = get_dataloader(file_path='./data_ptb-xl', batch_size=64, mode='val', device=device, _lang=language)
    
n_epochs=50
encoder_hidden_size = 256
decoder_hidden = [128, 256, 512, 1024, 2048]
criterion = nn.NLLLoss()

for decoder_hidden_size in decoder_hidden:
    encoder = NN(num_leads=12,
                 hidden_size=encoder_hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size=decoder_hidden_size,
                             encoder_hidden_size=encoder_hidden_size,
                             output_size=language.n_words,
                             max_len=language.max_len).to(device)
    
    print(f"############################ decoder hidden size: {decoder_hidden_size} ############################")
    print(f"encoder #parameters: {count_parameters(encoder)}")
    print(f"decoder #parameters: {count_parameters(decoder)}\n")

    train(dataloader, val_dataloader, encoder, decoder, criterion, language, n_epochs, size=decoder_hidden_size)

############################ decoder hidden size: 128 ############################
encoder #parameters: 6545152
decoder #parameters: 996325
0m 25s (- 20m 53s) (1 2.0%) | Train Loss: 0.9623 | Val METEOR: 0.3209
0m 51s (- 20m 30s) (2 4.0%) | Train Loss: 0.3397 | Val METEOR: 0.3363
1m 16s (- 20m 4s) (3 6.0%) | Train Loss: 0.2738 | Val METEOR: 0.3717
1m 42s (- 19m 40s) (4 8.0%) | Train Loss: 0.2451 | Val METEOR: 0.3977
2m 8s (- 19m 16s) (5 10.0%) | Train Loss: 0.2251 | Val METEOR: 0.3543
2m 34s (- 18m 51s) (6 12.0%) | Train Loss: 0.2109 | Val METEOR: 0.4367
3m 0s (- 18m 26s) (7 14.0%) | Train Loss: 0.1999 | Val METEOR: 0.3968
3m 25s (- 18m 1s) (8 16.0%) | Train Loss: 0.1911 | Val METEOR: 0.4092
3m 51s (- 17m 35s) (9 18.0%) | Train Loss: 0.183 | Val METEOR: 0.4392
4m 17s (- 17m 11s) (10 20.0%) | Train Loss: 0.177 | Val METEOR: 0.4325
4m 43s (- 16m 45s) (11 22.0%) | Train Loss: 0.1715 | Val METEOR: 0.4386
5m 9s (- 16m 19s) (12 24.0%) | Train Loss: 0.1668 | Val METEOR: 0.4308
5m 35s (- 15m 54

### Test

In [10]:
torch.manual_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

encoder_hidden_size = 256
decoder_hidden = [128, 256, 512, 1024, 2048]
criterion = nn.NLLLoss()

_, test_dataloader = get_dataloader(file_path='./data_ptb-xl', batch_size=64, mode='test', device=device, _lang=language)
results = []

for decoder_hidden_size in decoder_hidden:
    encoder = NN(num_leads=12,
                 hidden_size=encoder_hidden_size).to(device)
    decoder = AttnDecoderRNN(hidden_size=decoder_hidden_size,
                             encoder_hidden_size=encoder_hidden_size,
                             output_size=language.n_words,
                             max_len=language.max_len).to(device)
    
    encoder.load_state_dict(torch.load(f'./models/m04_EcgToText_ISIBrnoAIMT/models_with_different_hidden_size/varying_decoder/Encoder_{decoder_hidden_size}.pth'))
    decoder.load_state_dict(torch.load(f'./models/m04_EcgToText_ISIBrnoAIMT/models_with_different_hidden_size/varying_decoder/Decoder_{decoder_hidden_size}.pth'))
    
    total_loss, f1, jaccard, rouge, meteor = validate_epoch(test_dataloader, encoder, decoder, criterion, language)

    results.append({
        "Decoder": f"Decoder {decoder_hidden_size}",
        "Encoder #parameters": count_parameters(encoder),
        "Decoder #parameters": count_parameters(decoder),
        "Test Loss": round(total_loss, 4),
        "Rouge-1 (p)": round(rouge["rouge-1"]["p"], 3),
        "Rouge-1 (r)": round(rouge["rouge-1"]["r"], 3),
        "Rouge-1 (f1)": round(rouge["rouge-1"]["f"], 3),
        "Rouge-2 (p)": round(rouge["rouge-2"]["p"], 3),
        "Rouge-2 (r)": round(rouge["rouge-2"]["r"], 3),
        "Rouge-2 (f1)": round(rouge["rouge-2"]["f"], 3),
        "Rouge-L (p)": round(rouge["rouge-l"]["p"], 3),
        "Rouge-L (r)": round(rouge["rouge-l"]["r"], 3),
        "Rouge-L (f1)": round(rouge["rouge-l"]["f"], 3),
        "METEOR": round(meteor, 3)
    })

df_results = pd.DataFrame(results)
df_results

,Decoder,Encoder #parameters,Decoder #parameters,Test Loss,Rouge-1 (p),Rouge-1 (r),Rouge-1 (f1),Rouge-2 (p),Rouge-2 (r),Rouge-2 (f1),Rouge-L (p),Rouge-L (r),Rouge-L (f1),METEOR
0,Decoder 128,6545152,996325,3.2047,0.590,0.607,0.574,0.451,0.456,0.433,0.587,0.604,0.570,0.504
1,Decoder 256,6545152,2219237,3.4594,0.592,0.602,0.572,0.449,0.453,0.431,0.588,0.597,0.568,0.501
2,Decoder 512,6545152,5353189,3.5877,0.573,0.563,0.543,0.426,0.414,0.400,0.568,0.559,0.538,0.476
3,Decoder 1024,6545152,14373605,2.7042,0.425,0.566,0.448,0.289,0.375,0.297,0.424,0.564,0.447,0.284
4,Decoder 2048,6545152,43424485,3.6610,0.510,0.529,0.494,0.356,0.361,0.338,0.507,0.526,0.491,0.399
